<a href="https://colab.research.google.com/github/deepak10281/mobile-sales-analysis-using-powerBi/blob/main/NBs/xinfinity_agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Import Library

In [ ]:
import pandas
import os
import pandas as pd
import faiss
from uuid import uuid4
from langchain_core.documents import Document
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_community.vectorstores import FAISS
from langchain_openai import AzureOpenAIEmbeddings, AzureChatOpenAI
from langchain_community.utilities import SQLDatabase
%load_ext autoreload
%autoreload 2
from dotenv import load_dotenv
load_dotenv(r'C:\Users\VR679RX\OneDrive - EY\Documents\Projects\Projects - 2024\LangGraph_Academy\gpt4o.env')

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


True

In [ ]:
db = SQLDatabase.from_uri("sqlite:///xinfinity_agent.db")

In [ ]:
# Print the dialect and available tables
print(db.dialect)
print(db.get_usable_table_names())

sqlite
['billing_table', 'customer_service_table', 'customer_table']


In [ ]:
from typing_extensions import TypedDict


class State(TypedDict):
    question: str
    query: str
    result: str
    answer: str

In [ ]:
llm = AzureChatOpenAI(temperature=0,
                           api_key=os.getenv('AZURE_OPENAI_API_KEY'),
                           azure_endpoint=os.getenv('AZURE_OPENAI_ENDPOINT'),
                           openai_api_version=os.getenv('AZURE_OPENAI_VERSION'),
                           azure_deployment=os.getenv('AZURE_GPT4O_MODEL')
                           )

embeddings = AzureOpenAIEmbeddings(
                            api_key=os.getenv('AZURE_OPENAI_API_KEY'),
                            azure_endpoint=os.getenv('AZURE_OPENAI_ENDPOINT'),
                            azure_deployment=os.getenv('AZURE_OPENAI_EMBEDDINGS_MODEL'),
                            openai_api_version=os.getenv('AZURE_OPENAI_VERSION'),
                            chunk_size = 1
                            )

### ReAct SQL Agent

In [ ]:
from langchain_community.agent_toolkits import SQLDatabaseToolkit

toolkit = SQLDatabaseToolkit(db=db, llm=llm)

tools = toolkit.get_tools()

tools

[QuerySQLDatabaseTool(description="Input to this tool is a detailed and correct SQL query, output is a result from the database. If the query is not correct, an error message will be returned. If an error is returned, rewrite the query, check the query, and try again. If you encounter an issue with Unknown column 'xxxx' in 'field list', use sql_db_schema to query the correct table fields.", db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x0000021271FA3A10>),
 InfoSQLDatabaseTool(description='Input to this tool is a comma-separated list of tables, output is the schema and sample rows for those tables. Be sure that the tables actually exist by calling sql_db_list_tables first! Example Input: table1, table2, table3', db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x0000021271FA3A10>),
 ListSQLDatabaseTool(db=<langchain_community.utilities.sql_database.SQLDatabase object at 0x0000021271FA3A10>),
 QuerySQLCheckerTool(description='Use this tool to 

In [ ]:
from langchain import hub

prompt_template = hub.pull("langchain-ai/sql-agent-system-prompt")

assert len(prompt_template.messages) == 1
prompt_template.messages[0].prompt.template = """You are an intelligent assistant helping users explore a customer billing system.\nGiven an input question, create a syntactically correct {dialect} query to run, then look at the results of the query and return the answer.\nUnless the user specifies a specific number of examples they wish to obtain, always limit your query to at most {top_k} results.\nYou can order the results by a relevant column to return the most interesting examples in the database.\nNever query for all the columns from a specific table, only ask for the relevant columns given the question.\nYou have access to tools for interacting with the database.\nOnly use the below tools. Only use the information returned by the below tools to construct your final answer.\nYou MUST double check your query before executing it. If you get an error while executing a query, rewrite the query and try again.\n\nDO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.) to the database.\n\nTo start you should ALWAYS look at the tables in the database to see what you can query.\nDo NOT skip this step.\nThen you should query the schema of the most relevant tables. \nBefore executing a query, make sure you have:
1. The correct **table** to query.
2. All **mandatory fields** for that table.
3. Clear disambiguation if needed (e.g., same name, multiple services).

It is important that You ask follow-up questions to gather missing fields based on this schema:
- `customer_table` needs customer_id.
- `customer_service_table` needs customer_id + service_id.
- `billing_table` needs customer_id + service_id (+ optional billing_date, status, plan_name).

When ready, construct a SQL-like query or structured representation to fetch the data."""
prompt_template.messages[0].pretty_print()

================================ System Message ================================

You are an intelligent assistant helping users explore a customer billing system.
Given an input question, create a syntactically correct {dialect} query to run, then look at the results of the query and return the answer.
Unless the user specifies a specific number of examples they wish to obtain, always limit your query to at most {top_k} results.
You can order the results by a relevant column to return the most interesting examples in the database.
Never query for all the columns from a specific table, only ask for the relevant columns given the question.
You have access to tools for interacting with the database.
Only use the below tools. Only use the information returned by the below tools to construct your final answer.
You MUST double check your query before executing it. If you get an error while executing a query, rewrite the query and try again.

DO NOT make any DML statements (INSERT, UPDATE, D

In [ ]:
system_message = prompt_template.format(dialect="SQLite", top_k=10)

In [ ]:
from langchain_core.messages import HumanMessage
from langgraph.prebuilt import create_react_agent

sql_agent_executor = create_react_agent(model= llm,
                                        tools= tools,
                                        name= 'sql_agent_executor',
                                        prompt= system_message)

In [ ]:
# question = "How much did I spend in the last 6 months?"

# for step in sql_agent_executor.stream(
#     {"messages": [{"role": "user", "content": question}]},
#     stream_mode="values",
# ):
#     step["messages"][-1].pretty_print()

### Create RAG Agent

In [ ]:
from langchain.tools.retriever import create_retriever_tool

In [ ]:
# from DataProcessing.DocumentParser import DocumentParser
# from IndexGeneration.DocumentChunk import DocumentChunk

# parser = DocumentParser()
# chunker = DocumentChunk()

# dataframe = pd.DataFrame()
# for pdf in ['HOW5220_DG558_XB6_getting_started.pdf','RSA_01012025.pdf']:
#     result = parser.extract_text(pdf,type='local')
#     chunk_df = chunker.process_chunk(pd.DataFrame(result),1500)
#     chunk_df['source'] = pdf
#     dataframe = pd.concat([chunk_df,dataframe])

# dataframe = dataframe.reset_index(drop=True)

# documents = []
# for _,row in dataframe.iterrows():
#     documents.append(
#         Document(page_content=row['merged_content'],
#                  metadata={
#                      'source':row['source'],
#                      'page_range':row['page_range'],
#                      'chunk_coordinates':row['chunk_coordinates']
#                  })
#     )

# index = faiss.IndexFlatL2(len(embeddings.embed_query("hello world")))
# vector_store = FAISS(
#     embedding_function=embeddings,
#     index=index,
#     docstore=InMemoryDocstore(),
#     index_to_docstore_id={},
# )
# uuids = [str(uuid4()) for _ in range(len(documents))]
# vector_store.add_documents(documents=documents, ids=uuids)
# vector_store.save_local('vector_store_index')

In [ ]:
vector_store = FAISS.load_local(
    "vector_store_index", embeddings, allow_dangerous_deserialization=True
)


In [58]:
# from DataProcessing.DocumentParser import DocumentParser
# from IndexGeneration.DocumentChunk import DocumentChunk

# parser = DocumentParser()
# chunker = DocumentChunk()

# dataframe = pd.DataFrame()
# for pdf in ['HOW5220_DG558_XB6_getting_started.pdf','RSA_01012025.pdf']:
#     result = parser.extract_text(pdf,type='local')
#     chunk_df = chunker.process_chunk(pd.DataFrame(result),1500)
#     chunk_df['source'] = pdf
#     dataframe = pd.concat([chunk_df,dataframe])

# dataframe = dataframe.reset_index(drop=True)

# documents = []
# for _,row in dataframe.iterrows():
#     documents.append(
#         Document(page_content=row['merged_content'],
#                  metadata={
#                      'source':row['source'],
#                      'page_range':row['page_range'],
#                      'chunk_coordinates':row['chunk_coordinates']
#                  })
#     )

# index = faiss.IndexFlatL2(len(embeddings.embed_query("hello world")))
# vector_store = FAISS(
#     embedding_function=embeddings,
#     index=index,
#     docstore=InMemoryDocstore(),
#     index_to_docstore_id={},
# )
# uuids = [str(uuid4()) for _ in range(len(documents))]
# vector_store.add_documents(documents=documents, ids=uuids)
# vector_store.save_local('vector_store_index')

# The above code is commented out, preventing the definition of `vector_store`.
# Uncomment the following lines to load the existing vector store:

from langchain_community.vectorstores import FAISS
from langchain_openai import AzureOpenAIEmbeddings # Make sure to import AzureOpenAIEmbeddings
import os

embeddings = AzureOpenAIEmbeddings(
                            api_key=os.getenv('AZURE_OPENAI_API_KEY'),
                            azure_endpoint=os.getenv('AZURE_OPENAI_ENDPOINT'),
                            azure_deployment=os.getenv('AZURE_OPENAI_EMBEDDINGS_MODEL'),
                            openai_api_version=os.getenv('AZURE_OPENAI_VERSION'),
                            chunk_size = 1
                            )

vector_store = FAISS.load_local(
    "vector_store_index", embeddings, allow_dangerous_deserialization=True
)

In [59]:
from langchain_core.prompts import ChatPromptTemplate

system_message = ChatPromptTemplate([
    ("system", "You are helpful assitant. You need to help user to answer query related to company's policies guidelines"),
     "{messages}"
])

In [ ]:
!pip install --upgrade langgraph langchain

from langgraph.prebuilt import create_react_agent

# Assuming 'vector_store' is your vector store instance
retriever_tool = create_retriever_tool(
    vector_store,
    name="search_company_docs",
    description="Useful for when you need to answer questions about company's policies and guidelines. Input should be a search query."
)

# Pass the 'llm' to the 'model' argument instead of 'llm_chain'
rag_react_agent = create_react_agent(
    model=llm,  # Changed from 'llm_chain' to 'model'
    tools=[retriever_tool],
    name='rag_agent_executor',
    prompt=system_message
)

In [71]:
question = 'what is liability for the damage?'
rag_react_agent.invoke({"messages": [{"role": "user", "content": question}]})

PermissionDeniedError: <html>
  <head>
    <style global>body{font-family:Arial,Helvetica,sans-serif}.container{align-items:center;display:flex;flex-direction:column;gap:2rem;height:100%;justify-content:center;width:100%}@keyframes enlarge-appear{0%{opacity:0;transform:scale(75%) rotate(-90deg)}to{opacity:1;transform:scale(100%) rotate(0deg)}}.logo{color:#8e8ea0}.scale-appear{animation:enlarge-appear .4s ease-out}@media (min-width:768px){.scale-appear{height:48px;width:48px}}.data:empty{display:none}.data{border-radius:5px;color:#8e8ea0;text-align:center}@media (prefers-color-scheme:dark){body{background-color:#343541}.logo{color:#acacbe}}</style>
  <meta http-equiv="refresh" content="360">
</head>
  <body>
    <div class="container">
      <div class="logo">
        <svg
          width="41"
          height="41"
          viewBox="0 0 41 41"
          fill="none"
          xmlns="http://www.w3.org/2000/svg"
          strokeWidth="2"
          class="scale-appear"
        >
          

In [75]:
from langchain.chat_models import AzureChatOpenAI
from langchain.agents import Tool, AgentExecutor # Import necessary modules


# Load LLM, using AzureChatOpenAI instead of ChatOpenAI
llm = AzureChatOpenAI(
    azure_deployment=os.getenv('AZURE_GPT4O_MODEL'),
    temperature=0,
    api_key=os.getenv('AZURE_OPENAI_API_KEY'),
    azure_endpoint=os.getenv('AZURE_OPENAI_ENDPOINT'),
    openai_api_version=os.getenv('AZURE_OPENAI_VERSION'),
)

# Define retriever - This was missing
# Replace with your actual retriever initialization
# Assuming you're using the vector_store defined earlier
from langchain.chains import RetrievalQA  # Import RetrievalQA

retriever = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=vector_store.as_retriever())

# Define tool
retriever_tool = Tool(
    name="legal_doc_search",
    func=retriever.run,  # Use retriever.run instead of retriever.invoke
    description="Useful for answering questions about legal document content"
)

# Create agent
agent = create_react_agent(
    model=llm,
    tools=[retriever_tool],
    name="rag_agent",
    prompt=system_message  # Optional, or use default
)

# Executor
agent_executor = AgentExecutor(agent=agent, tools=[retriever_tool], verbose=True)

# Ask your question
response = agent_executor.invoke({"input": "What is liability for the damage?"})
print(response)

ValidationError: 1 validation error for VectorStoreRetriever
vectorstore
  Input should be an instance of VectorStore [type=is_instance_of, input_value=<langchain_community.vect...bject at 0x7c35c3f56ad0>, input_type=FAISS]
    For further information visit https://errors.pydantic.dev/2.11/v/is_instance_of

In [76]:
from langchain.chat_models import AzureChatOpenAI
from langchain.agents import Tool, AgentExecutor # Import necessary modules
from langchain.chains import RetrievalQA  # Import RetrievalQA

# Load LLM, using AzureChatOpenAI instead of ChatOpenAI
llm = AzureChatOpenAI(
    azure_deployment=os.getenv('AZURE_GPT4O_MODEL'),
    temperature=0,
    api_key=os.getenv('AZURE_OPENAI_API_KEY'),
    azure_endpoint=os.getenv('AZURE_OPENAI_ENDPOINT'),
    openai_api_version=os.getenv('AZURE_OPENAI_VERSION'),
)

# Define retriever - This was missing
# Replace with your actual retriever initialization
# Assuming you're using the vector_store defined earlier
# Instead of using vector_store.as_retriever(), pass vector_store directly
retriever = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=vector_store.as_retriever(), vectorstore=vector_store)

# Define tool
retriever_tool = Tool(
    name="legal_doc_search",
    func=retriever.run,  # Use retriever.run instead of retriever.invoke
    description="Useful for answering questions about legal document content"
)

# Create agent
agent = create_react_agent(
    model=llm,
    tools=[retriever_tool],
    name="rag_agent",
    prompt=system_message  # Optional, or use default
)

# Executor
agent_executor = AgentExecutor(agent=agent, tools=[retriever_tool], verbose=True)

# Ask your question
response = agent_executor.invoke({"input": "What is liability for the damage?"})
print(response)

ValidationError: 1 validation error for VectorStoreRetriever
vectorstore
  Input should be an instance of VectorStore [type=is_instance_of, input_value=<langchain_community.vect...bject at 0x7c35c3f56ad0>, input_type=FAISS]
    For further information visit https://errors.pydantic.dev/2.11/v/is_instance_of

## Supervisor Agent

In [ ]:
from langgraph_supervisor import create_supervisor

In [ ]:
workflow = create_supervisor(
    [sql_agent_executor, rag_react_agent],
    model=llm,
    prompt=(
        "You are a team supervisor managing a customer billing sql analysis expert and a math expert. "
        "For customer billing realted questions, use sql_agent_executor. "
        "For queries related to company's policies and guidelines, use rag_agent_executor."
        "Avoid mentioning any agent name in your response"
    )
)

In [ ]:
app = workflow.compile()
result = app.invoke({
    "messages": [
        {
            "role": "user",
            "content": 'what is liability for the damage?'
        }
    ]
})

In [72]:
result

NameError: name 'result' is not defined